In [41]:
%matplotlib inline

import os

import pandas as pd
import seaborn as sns
import numpy as np
import scipy.stats as stats

import matplotlib as mpl
import matplotlib.pyplot as plt

from skimage import exposure 
from skimage.exposure import match_histograms 

import cv2

Creating separate arrays for different channels

In [42]:
pos_R_data=np.array([])
pos_G_data=np.array([])
pos_B_data=np.array([])
neg_R_data=np.array([])
neg_G_data=np.array([])
neg_B_data=np.array([])
R_data=np.array([])
G_data=np.array([])
B_data=np.array([])

In [43]:
directory = 'negative'
i=0
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    f = os.path.join(directory, filename)  
    bacteria = cv2.imread(f)
    neg_R_data=np.append(neg_R_data,[np.array(bacteria[:,:,0].flatten())])
    neg_G_data=np.append(neg_G_data,[np.array(bacteria[:,:,1].flatten())])
    neg_B_data=np.append(neg_B_data,[np.array(bacteria[:,:,2].flatten())])
    #print(i)
    i+=1

directory = 'positive1'
i=0
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    f = os.path.join(directory, filename)     
    bacteria = cv2.imread(f)
    pos_R_data=np.append(pos_R_data,[np.array(bacteria[:,:,0].flatten())])
    pos_G_data=np.append(pos_G_data,[np.array(bacteria[:,:,1].flatten())])
    pos_B_data=np.append(pos_B_data,[np.array(bacteria[:,:,2].flatten())])
    #print(i)
    i+=1

R_data=np.append(neg_R_data,pos_R_data)
G_data=np.append(neg_G_data,pos_G_data)
B_data=np.append(neg_B_data,pos_B_data)
'''
print(neg_R_data) 
print(np.size(neg_R_data)) 
print(pos_R_data) 
print(np.size(pos_R_data)) 
print(R_data) 
print(np.size(R_data)) 
'''

'\nprint(neg_R_data) \nprint(np.size(neg_R_data)) \nprint(pos_R_data) \nprint(np.size(pos_R_data)) \nprint(R_data) \nprint(np.size(R_data)) \n'

In [44]:
neg_R_mean=np.mean(neg_R_data)
neg_G_mean=np.mean(neg_G_data)
neg_B_mean=np.mean(neg_B_data)
pos_R_mean=np.mean(pos_R_data)
pos_G_mean=np.mean(pos_G_data)
pos_B_mean=np.mean(pos_B_data)
R_mean=np.mean(R_data)
G_mean=np.mean(G_data)
B_mean=np.mean(B_data)

R_com=''
G_com=''
B_com=''
if (neg_R_mean<R_mean):
    R_com='NEG IS LESS'
else:
    R_com='POS IS LESS'
if (neg_G_mean<G_mean):
    G_com='NEG IS LESS'
else:
    G_com='POS IS LESS'
if (neg_B_mean<B_mean):
    B_com='NEG IS LESS'
else:
    B_com='POS IS LESS'

Consider the mean for different channels and types (G-/G+)

In [45]:
data = {
    'neg_mean': [neg_R_mean, neg_G_mean, neg_B_mean],
    'pos_mean': [pos_R_mean, pos_G_mean, pos_B_mean],
    'common_mean': [R_mean, G_mean, B_mean],
    'comparison': [R_com,G_com,B_com]
}
row_labels = ['R', 'G','B']

df = pd.DataFrame(data=data, index=row_labels)
df

,neg_mean,pos_mean,common_mean,comparison
R,132.644588,152.904086,143.695223,NEG IS LESS
G,210.259394,198.065795,203.608340,POS IS LESS
B,252.026905,232.955019,241.624058,POS IS LESS


Hypothesis: we can use the mean for channels in RGB color space to find out if our image shows G- or G+ microbiota (based on the table above)

In [46]:
def RGB_mean_test(nameline):
    global R_data
    global G_data
    global B_data
    bact = cv2.imread(nameline)
    #R_data=np.append(R_data,[np.array(bact[:,:,0].flatten())])
    #G_data=np.append(G_data,[np.array(bact[:,:,1].flatten())])
    #B_data=np.append(B_data,[np.array(bact[:,:,2].flatten())])

    curr_R_mean=np.mean([np.array(bact[:,:,0].flatten())])
    curr_G_mean=np.mean([np.array(bact[:,:,1].flatten())])
    curr_B_mean=np.mean([np.array(bact[:,:,2].flatten())])
    R_mean=np.mean(R_data)
    G_mean=np.mean(G_data)
    B_mean=np.mean(B_data)

    R_res=''
    G_res=''
    B_res=''
    if (curr_R_mean<R_mean):
        R_res='NEG'
    else:
        R_res='POS'
    if (curr_G_mean>G_mean):
        G_res='NEG'
    else:
        G_res='POS'
    if (curr_B_mean>B_mean):
        B_res='NEG'
    else:
        B_res='POS'

    data = {
    'current mean': [curr_R_mean, curr_G_mean, curr_B_mean],
    'common mean': [R_mean, G_mean, B_mean],
    'test result': [R_res,G_res,B_res]
    }
    row_labels = ['R-test', 'G-test','B-test']

    df = pd.DataFrame(data=data, index=row_labels)
    print(nameline)
    print(df)


Let's take a look on some tests

In [47]:
RGB_mean_test('NEG0.jpg')
RGB_mean_test('NEG1.jpg')
RGB_mean_test('POS0.jpg')
RGB_mean_test('POS1.jpg')

NEG0.jpg
        current mean  common mean test result
R-test    165.763781   143.695223         POS
G-test    195.545281   203.608340         POS
B-test    239.595305   241.624058         POS
NEG1.jpg
        current mean  common mean test result
R-test    125.486768   143.695223         NEG
G-test    207.361990   203.608340         NEG
B-test    253.430902   241.624058         NEG
POS0.jpg
        current mean  common mean test result
R-test    154.737813   143.695223         POS
G-test    194.218033   203.608340         POS
B-test    238.101061   241.624058         POS
POS1.jpg
        current mean  common mean test result
R-test    138.675455   143.695223         NEG
G-test    155.247334   203.608340         POS
B-test    199.625350   241.624058         POS


Perhaps we should use more data to find correct common means for further comparison

Perhaps the first table is incorrect due to a lack of data

Or the whole idea is incorrect